In [122]:
import pandas as pd
import pickle

# Reads the recipe links from a CSV generated by recipe_links_importer.py
senate_transactions = pd.read_csv('senate_transactions.csv')

# Unpickling dictionary of stock data
infile = open('ticker_data.pkl', 'rb')
ticker_data = pickle.load(infile)
infile.close()

print (ticker_data[next(iter(ticker_data))])

              open     high       low    close  adjusted close   volume  \
2011-01-03   13.74   13.770   13.4700   13.710          13.710   607600   
2011-01-04   13.77   13.840   13.1400   13.320          13.320   318200   
2011-01-05   13.33   13.700   13.3100   13.700          13.700   238100   
2011-01-06   13.67   13.770   13.5600   13.700          13.700   159200   
2011-01-07   13.76   13.830   13.5700   13.730          13.730   216500   
...            ...      ...       ...      ...             ...      ...   
2020-04-09  259.95  276.615  258.6000  275.270         275.270  1166685   
2020-04-13  279.88  285.790  270.1100  273.260         273.260  1190085   
2020-04-14  276.79  285.250  274.0549  280.670         280.670  1340746   
2020-04-15  278.90  300.000  277.6300  291.690         291.690  1266819   
2020-04-16  295.00  309.090  295.0000  306.275         306.275  1554222   

            dividend amount  split coefficient  
2011-01-03              0.0                1.0  
2

In [123]:
import datetime

#Creating a DataFrame of just public transactions
senate_transactions_public = senate_transactions[senate_transactions.ticker != '--'].reset_index(drop=True)
senate_transactions_public.reset_index(drop=True)

#Initializing list of T days after transaction
t_list = [0, 5, 10, 20, 30, 60, 90]
stock_list = []

for i in range(2):
    transaction_date = senate_transactions_public.iloc[i, senate_transactions_public.columns.get_loc('transaction_date')]
    ticker = senate_transactions_public.iloc[i, senate_transactions_public.columns.get_loc('ticker')]
    ticker_dataframe = ticker_data[ticker]
    transaction_date_datetime = datetime.datetime.strptime(transaction_date, '%Y-%m-%d').date()
    per_stock_list = []
    #Looping through each iteration of +T days after transaction
    for i in t_list:
        #Setting up days +/- 2 days of target transaction to account for weekends and holidays
        future_transaction_date_datetime = transaction_date_datetime + datetime.timedelta(days=i)
        future_transaction_date_datetime_minus_one = transaction_date_datetime + datetime.timedelta(days=(i-1))
        future_transaction_date_datetime_plus_one = transaction_date_datetime + datetime.timedelta(days=(i+1))
        future_transaction_date_datetime_minus_two = transaction_date_datetime + datetime.timedelta(days=(i-2))
        future_transaction_date_datetime_plus_two = transaction_date_datetime + datetime.timedelta(days=(i+2))    
        future_transaction_date = future_transaction_date_datetime.strftime("%Y-%m-%d")
        future_transaction_date_minus_one = future_transaction_date_datetime_minus_one.strftime("%Y-%m-%d")
        future_transaction_date_plus_one = future_transaction_date_datetime_plus_one.strftime("%Y-%m-%d")
        future_transaction_date_minus_two = future_transaction_date_datetime_minus_two.strftime("%Y-%m-%d")
        future_transaction_date_plus_two = future_transaction_date_datetime_plus_two.strftime("%Y-%m-%d")
        try:
            adj_closing_price = ticker_dataframe.loc[future_transaction_date, 'adjusted close']
        except:
            try:
                adj_closing_price = ticker_dataframe.loc[future_transaction_date_minus_one, 'adjusted close']
            except:
                try:
                    adj_closing_price = ticker_dataframe.loc[future_transaction_date_plus_one, 'adjusted close']
                except:
                    try:
                        adj_closing_price = ticker_dataframe.loc[future_transaction_date_minus_two, 'adjusted close']
                    except:
                        try:
                            adj_closing_price = ticker_dataframe.loc[future_transaction_date_plus_two, 'adjusted close']
                        except:
                            adj_closing_price = None
        per_stock_list.append(adj_closing_price)
        #print (adj_closing_price)
    stock_list.append(per_stock_list)
    
stock_data = pd.DataFrame(stock_list, columns=['Adj. Closing Price T+0', 'Adj. Closing Price T+5', 'Adj. Closing Price T+10', 'Adj. Closing Price T+20', 'Adj. Closing Price T+30', 'Adj. Closing Price T+60', 'Adj. Closing Price T+90'])

#display (stock_data)

senate_transactions_public_with_prices = pd.concat([senate_transactions_public, stock_data.reindex(senate_transactions_public.index)], axis=1)

display(senate_transactions_public_with_prices)

,Unnamed: 0,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,Adj. Closing Price T+0,Adj. Closing Price T+5,Adj. Closing Price T+10,Adj. Closing Price T+20,Adj. Closing Price T+30,Adj. Closing Price T+60,Adj. Closing Price T+90
0,4,2020-03-17,Spouse,DXCM,"DexCom, Inc.",Stock,Sale (Partial),"$250,001 - $500,000",--,Tina Smith,https://efdsearch.senate.gov/search/view/ptr/8...,198.74,210.3,262.96,269.31,306.275,None,None
1,5,2020-03-17,Spouse,PODD,Insulet Corporation,Stock,Sale (Partial),"$250,001 - $500,000",--,Tina Smith,https://efdsearch.senate.gov/search/view/ptr/8...,130.11,143.8,175.75,171.90,192.350,None,None
2,8,2020-03-11,Self,ILMN,"Illumina, Inc.",Stock,Sale (Full),"$1,001 - $15,000",--,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/ptr/4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,2020-03-11,Self,CGNX,Cognex Corporation,Stock,Sale (Full),"$15,001 - $50,000",--,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/ptr/4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,2020-03-11,Self,SIEGY,Siemens Aktiengesellschaft,Stock,Sale (Full),"$15,001 - $50,000",--,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/ptr/4...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6703,8599,2014-01-29,Spouse,C,"Citigroup, Inc. (NYSE)",NaN,Sale (Partial),"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6704,8600,2014-01-28,Spouse,EBAY,eBay Inc. (NASDAQ),NaN,Sale (Partial),"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6705,8601,2014-01-24,Spouse,FB,"Facebook, Inc. (NASDAQ)",NaN,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6706,8602,2014-01-24,Spouse,CRM,"Salesforce.com, Inc (NYSE)",NaN,Purchase,"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
